# Lifecycle Reports (BMC Discovery)

This notebook reproduces the DisMAL lifecycle exports using the raw CSVs generated by the CLI.
It reads `raw_exports/<appliance>/database_lifecycle_report.csv`, `*_os_lifecycle_report.csv`, and `*_software_lifecycle_report.csv` for preview and optional re-export to the standard output folders.


In [ ]:
# TODO: Fix report headers


This workflow consolidates the database, operating system, and software lifecycle reports from the pre-generated raw exports.


## Requirements

In [ ]:
# %pip install -q pandas pyyaml

import re
from pathlib import Path

import pandas as pd
import yaml


## Configuration (from config.yaml)

Reads settings from `../config.yaml` including target, token/token_file,
API version, and SSL verification preference.
Saves the CSV to `../output_<target>/credential_success.csv`.

In [ ]:
from pathlib import Path
import re
import yaml

def load_config_params(
    start: Path,
    appliance_name: str = None,
    appliance_index: int = 0,
) -> dict:
    def _find_repo_root(start: Path) -> Path:
        for p in [start] + list(start.parents):
            if (p / 'config.yaml').exists():
                return p
        return start.parent

    def _slugify(value: str) -> str:
        return re.sub(r'[^A-Za-z0-9]+', '_', value).strip('_').lower() or 'default'

    repo_root = _find_repo_root(start)
    config_path = repo_root / 'config.yaml'

    with open(config_path, 'r') as fh:
        cfg = yaml.safe_load(fh) or {}

    apps = cfg.get('appliances') or []
    selected = None
    if isinstance(apps, list) and apps:
        if appliance_name:
            selected = next((a for a in apps if a.get('name') == appliance_name), None)
            if selected is None:
                raise ValueError(f"No appliance named '{appliance_name}' in config.yaml")
        else:
            try:
                selected = apps[int(appliance_index)]
            except Exception:
                selected = apps[0]

    target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
    if not target:
        raise ValueError('config.yaml missing "target"')

    token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
    token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
    if not token and token_file:
        tf_path = Path(token_file)
        if not tf_path.is_absolute():
            tf_path = repo_root / tf_path
        with open(tf_path, 'r') as tf:
            token = tf.read().strip()
    if not token:
        token = None

    api_version = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
    verify_ssl = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

    sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
    output_dir = repo_root / f'output_{sanitized}'
    output_dir.mkdir(parents=True, exist_ok=True)

    export_name = ((selected or {}).get('name') or appliance_name or sanitized)
    raw_export_dir = repo_root / 'raw_exports' / _slugify(export_name)

    return {
        "repo_root": repo_root,
        "config_path": config_path,
        "cfg": cfg,
        "selected": selected,
        "target": target,
        "token": token,
        "api_version": api_version,
        "verify_ssl": verify_ssl,
        "output_dir": output_dir,
        "raw_export_dir": raw_export_dir,
    }


In [ ]:
def init_appliance(appliance_name: str = "prod"):
    params = load_config_params(Path.cwd(), appliance_name=appliance_name)

    target = params["target"]
    api_version = params["api_version"]
    verify_ssl = params["verify_ssl"]
    output_dir = params["output_dir"]
    raw_export_dir = params["raw_export_dir"]

    print('Appliance Name :', appliance_name)
    print('Target         :', target)
    print('API Version    :', api_version)
    print('Verify SSL     :', verify_ssl)
    print('Raw CSV folder :', raw_export_dir)
    print('Output folder  :', output_dir)

    return {
        "params": params,
        "target": target,
        "api_version": api_version,
        "verify_ssl": verify_ssl,
        "output_dir": output_dir,
        "raw_export_dir": raw_export_dir,
        "appliance_name": appliance_name,
    }


print('Initialise Prod:')
twprod = init_appliance('prod')

print('Initialise Dev:')
twdev = init_appliance('dev')


In [ ]:
print("Initialise Prod:")
twprod = init_appliance("prod")

print("Initialise Dev:")
twdev = init_appliance("dev")

In [ ]:
LIFECYCLE_EXPORTS = [
    {'title': 'Database Lifecycle Report', 'filename': 'database_lifecycle_report.csv', 'output': 'database_lifecycle'},
    {'title': 'OS Lifecycle Report', 'filename': 'os_lifecycle_report.csv', 'output': 'os_lifecycle'},
    {'title': 'Software Lifecycle Report', 'filename': 'software_lifecycle_report.csv', 'output': 'software_lifecycle'},
    {'title': 'Hardware Lifecycle Report', 'filename': 'hardware_eol_report.csv', 'output': 'hardware_lifecycle'},
]

BASE_EXPORT_COLUMNS = ['Appliance Target', 'Appliance Name', 'Query Title']

def load_lifecycle_export(instance, meta):
    csv_path = instance['raw_export_dir'] / meta['filename']
    if not csv_path.exists():
        raise FileNotFoundError(f"Missing export {csv_path}")

    df = pd.read_csv(csv_path)
    df = df.drop(columns=[c for c in BASE_EXPORT_COLUMNS if c in df.columns], errors='ignore')
    df.insert(0, 'Discovery Instance', instance['target'])
    return df

def preview_exports(instance):
    frames = {}
    for meta in LIFECYCLE_EXPORTS:
        print(f"=== {meta['title']} ===")
        df = load_lifecycle_export(instance, meta)
        frames[meta['output']] = df
        print(instance['target'])
        print(f"Rows returned: {len(df)}")
        if not df.empty:
            display(df.head(10))
    return frames

def save_exports(frames, instance):
    for meta in LIFECYCLE_EXPORTS:
        df = frames.get(meta['output'])
        if df is None:
            continue
        output_csv = instance['output_dir'] / f"{meta['output']}.csv"
        df.to_csv(output_csv, index=False)
        print(f"Saved to {output_csv}")


## Session and helpers

In [ ]:
prod_frames = preview_exports(twprod)
dev_frames = preview_exports(twdev)


In [ ]:
print('Saving lifecycle exports (prod)')
save_exports(prod_frames, twprod)
print('Saving lifecycle exports (dev)')
save_exports(dev_frames, twdev)


---
### Notes
- Ensure the raw export CSVs are refreshed before running the notebook so the lifecycle data remains current.
- Update `LIFECYCLE_EXPORTS` if new lifecycle queries are added to the export workflow.
- Previews are limited to the first 10 rows for readability; adjust `preview_exports` if you need more context.
